In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

df = pd.read_csv('Dados preliminares - Página1.csv')
df.rename(columns={'Sensor 1': 'Sensor Flexão'}, inplace=True)

#Separando os dados em treinamento e teste
train = df.iloc[:-30]
test = df.iloc[-30:]

#Alterando a ordenação das linhas
train = train.sample(frac=1.0, random_state=42)
test = test.sample(frac=1.0, random_state=42)

X_train = train.iloc[:, :3]
X_test = test.iloc[:, :3]
y_train = train['Posição']
y_test = test['Posição']

model = SVC()
model.fit(X_train, y_train)

# Convert the model to ONNX format
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onx = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("svc.onnx", "wb") as f:
    f.write(onx.SerializeToString())
